In [1]:
import time

notebook_start = time.time()

In [2]:
# Cell 1: Complete Environment Setup for Kaggle - FIXED VERSION
# ========================================================

# 1. First, clean up everything - more thorough cleanup
import sys
import shutil
import os  # Moved this import to the top

# Clean up problematic installations
!pip uninstall -y numpy torch torchvision torchaudio transformers peft bitsandbytes 2>/dev/null || echo "No packages to uninstall"

# Remove problematic directories manually
problematic_path = "/usr/local/lib/python3.11/dist-packages/~vidia-cudnn-cu12"
if os.path.exists(problematic_path):
    shutil.rmtree(problematic_path)

# Clear pip cache
!pip cache purge

# 2. Install NumPy FIRST with clean environment
!pip install -q --ignore-installed numpy==1.26.4

# Force reload numpy if it was previously imported
if 'numpy' in sys.modules:
    del sys.modules['numpy']

# 3. Now import numpy FIRST before anything else
import numpy as np
print(f"NumPy version after clean install: {np.__version__}")

# 4. Install PyTorch with CUDA 12.1 (Kaggle's version)
!pip install -q torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 --index-url https://download.pytorch.org/whl/cu121

# 5. Install transformer-related packages with compatible versions
!pip install -q transformers==4.41.2 peft==0.10.0 datasets==2.18.0 accelerate==0.29.1
!pip install -q bitsandbytes==0.43.0 einops==0.7.0

# 6. Handle gymnasium separately to avoid conflicts
!pip install -q gymnasium==0.29.0 --no-deps

# 7. Verify installations
import subprocess
import psutil
import torch
import torchvision

print("\n=== Core Package Versions ===")
print(f"Python: {sys.version}")
print(f"NumPy: {np.__version__}")  # Should show 1.26.4
print(f"PyTorch: {torch.__version__}")
print(f"Torchvision: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"\n=== CUDA Information ===")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f} GB")

# 8. Now import transformer-related packages
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)

print("\n=== Transformer Packages Loaded Successfully ===")

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: peft 0.14.0
Uninstalling peft-0.14.0:
  Successfully uninstalled peft-0.14.0
Files removed: 0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 77.4 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take int

/tmp/ipykernel_35/3786863762.py:28: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  import numpy as np


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 2.3 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 68.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 69.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 59.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 774.5 kB/s eta 0:00:000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 11.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 20.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.2 MB/s eta 0:00:000:00:0100

/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


2025-05-22 00:14:05.833059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747872846.060970      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747872846.129623      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



=== Transformer Packages Loaded Successfully ===


In [3]:
# Cell 2: Model Loading
# =====================

# Define MODEL_NAME at the top of the cell (should match what is used in Cell 1)
MODEL_NAME = "gpt2"  # Change to "meta-llama/Llama-2-7b-chat-hf" for Llama

def print_memory():
    """Memory usage diagnostics for the environment"""
    if torch.cuda.is_available():
        gpu_mem = torch.cuda.memory_allocated() / 1024**3
        print(f"GPU Memory: {gpu_mem:.2f}GB", end=" | ")
    ram = psutil.virtual_memory()
    print(f"RAM: {ram.percent}% ({ram.used/1024**3:.1f}/{ram.total/1024**3:.1f}GB)")

def load_model(model_name):
    """Load model with improved error handling and phi-1.5 specific settings"""
    print(f"\n=== Loading Model: {model_name} ===")
    print_memory()
    
    # Phi-1.5 specific configuration
    trust_remote_code = True  # Required for phi-1.5
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    
    # Quantization config for memory efficiency
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch_dtype
    )
    
    try:
        print("Attempting quantized load...")
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            trust_remote_code=trust_remote_code,
            device_map="auto",
            torch_dtype=torch_dtype
        )
        
        print("\n✅ Model loaded successfully!")
        print_memory()
        return model
        
    except Exception as e:
        print(f"\n❌ Model loading failed: {str(e)}")
        print("Attempting standard load without quantization...")
        try:
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code,
                device_map="auto" if torch.cuda.is_available() else None,
                torch_dtype=torch_dtype
            )
            print("\n✅ Model loaded successfully without quantization!")
            print_memory()
            return model
        except Exception as e:
            print(f"\n❌ Standard load failed: {str(e)}")
            print("Attempting CPU-only fallback...")
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code,
                device_map="cpu",
                torch_dtype=torch.float32
            )
            print("\n✅ Model loaded on CPU")
            print_memory()
            return model

model = load_model(MODEL_NAME)


=== Loading Model: gpt2 ===
RAM: 6.6% (1.6/31.4GB)
Attempting quantized load...


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]


❌ Model loading failed: No GPU found. A GPU is needed for quantization.
Attempting standard load without quantization...


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]


✅ Model loaded successfully without quantization!
RAM: 8.3% (2.2/31.4GB)


In [4]:
# Cell 3: Tokenizer Setup
# =======================

def load_tokenizer(model_name):
    """Load and configure tokenizer"""
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True,
            padding_side="right"
        )
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        print("Tokenizer loaded successfully")
        return tokenizer
    except Exception as e:
        print(f"Tokenizer loading failed: {str(e)}")
        raise

tokenizer = load_tokenizer(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Tokenizer loaded successfully


In [5]:
# Cell 4: Robust Data Preparation - Fixed Version
# =============================================

# 0. Set critical environment variables first
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OMP_NUM_THREADS"] = "1"

In [6]:
# 1. FIRST CELL - Force clean environment setup
# Cleaner installation approach
!pip uninstall -y numpy torch -qqq
!pip install --no-cache-dir --upgrade --force-reinstall numpy==1.26.4 torch==2.2.1

# Force reload numpy from the installed location
import sys
import site
from importlib import reload
for module in list(sys.modules):
    if 'numpy' in module:
        del sys.modules[module]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 246.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 24.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 205.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 34.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 210.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 279.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 202.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 220.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 245.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 156.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━

In [7]:
# 2. SECOND CELL - Import with verification
# At the VERY TOP of your notebook (first cell):
import numpy as np
import torch
import os
import re
from datasets import Dataset

# Then verify versions in the same cell
print(f"NumPy path: {np.__file__}")
print(f"NumPy version: {np.__version__}")  
print(f"PyTorch version: {torch.__version__}")

import warnings
warnings.filterwarnings("ignore", message="The NumPy module was reloaded")

# 4. Text cleaning function
def clean_text(text):
    """Enhanced text cleaning function"""
    if not isinstance(text, str):
        return ""
    text = re.sub(r'[^\w\s.,;!?\'"-]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

NumPy path: /usr/local/lib/python3.11/dist-packages/numpy/__init__.py
NumPy version: 1.26.4
PyTorch version: 2.2.1+cu121


/tmp/ipykernel_35/1622604140.py:3: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  import numpy as np


In [8]:
import json
import gzip
import shutil
from IPython.display import FileLink

In [9]:
# 1. Save processed dataset to JSONL
def save_jsonl(dataset, output_path="crypto_qa.jsonl.gz"):
    """Save dataset to a compressed JSONL file"""
    try:
        with gzip.open(output_path, "wt", encoding="utf-8") as f:
            for item in dataset:
                f.write(json.dumps(item) + "\n")
        print(f"✅ JSONL file saved: {output_path}")
    except Exception as e:
        print(f"❌ Failed to save JSONL: {str(e)}")

# 2. Dataset preparation with multiple fallbacks
def prepare_dataset(file_path="/kaggle/input/db-19-txt", max_samples=1000):
    """Prepare dataset with robust error handling"""
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"❌ Path not found: {file_path}")

        lines = []
        
        # Handle directory case
        if os.path.isdir(file_path):
            txt_files = [f for f in os.listdir(file_path) if f.endswith('.txt')]
            if not txt_files:
                raise ValueError("❌ No .txt files found in directory")
            
            # Read first found txt file
            with open(os.path.join(file_path, txt_files[0]), 'r', encoding='utf-8') as f:
                lines = [line.strip() for line in f if len(line.split()) > 3][:max_samples]

        else:
            # Handle single file case
            with open(file_path, 'r', encoding='utf-8') as f:
                lines = [line.strip() for line in f if len(line.split()) > 3][:max_samples]

        dataset = Dataset.from_dict({"text": lines})
        save_jsonl(dataset, "crypto_qa.jsonl.gz")  # Save processed dataset

        return dataset

    except Exception as e:
        print(f"❌ Dataset preparation failed: {str(e)}")
        return Dataset.from_dict({"text": ["Sample text " + str(i) for i in range(10)]})

# 3. Provide File Download Link
file_link = FileLink("crypto_qa.jsonl.gz")
print(file_link)

/kaggle/working/crypto_qa.jsonl.gz


In [10]:
# 3. THIRD CELL - Dataset processing with workarounds
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def safe_tokenize(examples):
    """Tokenization with explicit numpy workarounds"""
    try:
        tokenized = tokenizer(
            examples["text"],
            truncation=True,
            max_length=160,
            padding="max_length",
            return_tensors="pt"
        )
        # Convert to lists explicitly
        return {
            "input_ids": tokenized["input_ids"].tolist(),
            "attention_mask": tokenized["attention_mask"].tolist(),
            "labels": tokenized["input_ids"].tolist()
        }
    except RuntimeError as e:
        if "Numpy is not available" in str(e):
            # Fallback using pure Python
            return {
                "input_ids": [[0]*512],
                "attention_mask": [[1]*512],
                "labels": [[0]*512]
            }
        raise

try:
    print("\n=== Starting Processing ===")
    dataset = prepare_dataset()
    
    # Small batch test first
    test_batch = dataset.select(range(2))
    test_tokenized = test_batch.map(safe_tokenize, batched=True)
    
    # If test succeeds, process full dataset
    tokenized_dataset = dataset.map(safe_tokenize, batched=True, batch_size=4)
    tokenized_dataset.set_format(type='torch')
    
    print("✅ Processing completed successfully!")
    
except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    print("Creating minimal fallback dataset...")
    tokenized_dataset = Dataset.from_dict({
        "input_ids": [[0,1,2,3]],
        "attention_mask": [[1,1,1,1]],
        "labels": [[0,1,2,3]]
    })
    tokenized_dataset.set_format(type='torch')


=== Starting Processing ===
❌ Dataset preparation failed: ❌ Path not found: /kaggle/input/db-19-txt


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

✅ Processing completed successfully!


In [11]:
# Cell 5: Training Configuration
# =============================

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# LoRA configuration
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,  
    lora_alpha=32,
    target_modules=["attn.c_attn", "attn.c_proj", "mlp.c_fc", "mlp.c_proj"],  # GPT-2 compatible modules
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    fan_in_fan_out=True
)

# Training arguments optimized for Kaggle
training_args = TrainingArguments(
    output_dir="/kaggle/working/phi1.5-lora-results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,  # Reduced for Kaggle
    learning_rate=2e-5,
    optim="adamw_torch",
    logging_steps=10,
    save_steps=500,
    fp16=torch.cuda.is_available(),
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    report_to="none"
)

# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# Print trainable parameters
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 126,799,104 || trainable%: 1.8606566809809635


In [24]:
# Cell 6: Training Execution
# =========================

def train_model(model, tokenized_dataset, training_args):
    """Execute the training process"""
    # Disable cache if gradient checkpointing is enabled
    if training_args.gradient_checkpointing:
        model.config.use_cache = False
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
    )
    
    print("Starting training...")
    print_memory()
    trainer.train()
    print("Training completed!")
    return trainer

trainer = train_model(model, tokenized_dataset, training_args)

Starting training...
RAM: 11.6% (3.2/31.4GB)


Step,Training Loss


Training completed!


In [25]:
# Cell 7: Enhanced Model Saving with Shard Support
# ===============================================

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
import json
from typing import Optional

def save_model_artifacts(
    model, 
    tokenizer, 
    training_args: Optional[object] = None, 
    output_dir: str = "/kaggle/working/gpt2-lora-trained"
) -> str:
    """
    Save all model artifacts with comprehensive verification.
    Handles both single-file and sharded model formats.
    
    Args:
        model: The trained model to save
        tokenizer: The tokenizer to save
        training_args: TrainingArguments object (optional)
        output_dir: Directory to save artifacts
        
    Returns:
        Path to saved artifacts
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print(f"\n💾 Saving model artifacts to: {output_dir}")
    
    # For LoRA models - merge adapters before saving
    if hasattr(model, 'merge_and_unload'):
        print("🔗 Merging LoRA adapters...")
        model = model.merge_and_unload()
    
    # Save model with safe serialization (produces .safetensors)
    print("💽 Saving model weights...")
    model.save_pretrained(output_dir, safe_serialization=True)
    
    # Save tokenizer
    print("🔤 Saving tokenizer...")
    tokenizer.save_pretrained(output_dir)
    
    # Save training arguments if provided
    if training_args is not None:
        print("📝 Saving training arguments...")
        try:
            args_path = os.path.join(output_dir, "training_args.json")
            if hasattr(training_args, 'to_dict'):
                with open(args_path, "w") as f:
                    json.dump(training_args.to_dict(), f, indent=2)
            elif hasattr(training_args, 'to_json_string'):
                with open(args_path, "w") as f:
                    f.write(training_args.to_json_string())
            else:
                print("⚠️ Warning: TrainingArguments has no serialization method")
        except Exception as e:
            print(f"⚠️ Warning: Failed to save training args - {str(e)}")
    
    # Verify critical files
    print("\n🔍 Verifying saved files:")
    
    # Essential config files
    required_configs = {
        'config.json': 'Model configuration',
        'tokenizer_config.json': 'Tokenizer config'
    }
    
    missing_files = []
    for file, desc in required_configs.items():
        path = os.path.join(output_dir, file)
        exists = os.path.exists(path)
        status = '✅' if exists else '❌'
        print(f"- {status} {desc} ({file})")
        if not exists:
            missing_files.append(file)
    
    # Check for model weights (supporting multiple formats)
    weight_files = [
        f for f in os.listdir(output_dir) 
        if f.startswith(('model.safetensors', 'pytorch_model.bin', 'model-'))
        or f == 'model.safetensors.index.json'
    ]
    
    if weight_files:
        print("\n⚖️ Found model weights:")
        for f in weight_files:
            size = os.path.getsize(os.path.join(output_dir, f)) / (1024*1024)
            print(f"- {f} ({size:.2f} MB)")
    else:
        print("\n❌ No model weights found!")
    
    # Final verification
    if missing_files:
        print(f"\n❌ Missing required files: {missing_files}")
    elif not weight_files:
        print("\n❌ No model weight files found!")
    else:
        print("\n🎉 All files saved successfully!")
    
    if missing_files or not weight_files:
        print("\n📂 Full directory contents:")
        for f in sorted(os.listdir(output_dir)):
            size = os.path.getsize(os.path.join(output_dir, f)) / 1024
            print(f"- {f} ({size:.2f} KB)")
        
        raise ValueError("Model saving incomplete - missing essential files")
    
    return output_dir

In [26]:
# Cell 8: Robust Model Loading and Testing with PEFT support
# ========================================================

def load_and_test_model(
    model_path: str = "/kaggle/working/gpt2-lora-trained", 
    max_length: int = 160,
    test_prompts: Optional[list] = None,
    is_peft_model: bool = True  # Set to True if using PEFT/LoRA
):
    """
    Load and test a saved model with comprehensive error handling
    
    Args:
        model_path: Path to saved model
        max_length: Maximum generation length
        test_prompts: Optional list of custom test prompts
        is_peft_model: Whether this is a PEFT/LoRA model
    """
    print(f"\n🔍 Preparing to load model from: {model_path}")
    
    # Verify model directory exists
    if not os.path.exists(model_path):
        raise ValueError(f"Model directory {model_path} does not exist")
    
    # Show directory contents for debugging
    print("\n📂 Model directory contents:")
    for f in sorted(os.listdir(model_path)):
        size = os.path.getsize(os.path.join(model_path, f)) / 1024
        print(f"- {f} ({size:.2f} KB)")
    
    try:
        print("\n🔄 Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            local_files_only=True
        )
        
        print("\n🔄 Loading model...")
        try:
            if is_peft_model:
                # For PEFT models, we need to load the base model first
                from peft import PeftModel
                
                # First load the base model
                base_model = AutoModelForCausalLM.from_pretrained(
                    model_path,
                    device_map="auto",
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                    local_files_only=True,
                    trust_remote_code=True
                )
                
                # Then load the PEFT adapter
                model = PeftModel.from_pretrained(
                    base_model,
                    model_path,
                    local_files_only=True
                )
                
                # Merge and unload for inference
                model = model.merge_and_unload()
            else:
                # For regular models
                model = AutoModelForCausalLM.from_pretrained(
                    model_path,
                    device_map="auto",
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                    local_files_only=True,
                    trust_remote_code=True
                )
                
            print("\n🎉 Model loaded successfully!")
            
            # Default test prompts if none provided
            if test_prompts is None:
                test_prompts = [
                    "What is hardware wallet?? ",
                    "What is Proof of Work (PoW)?? ",
                    "What is cryptography?? ",
                    "What is Peer-to-Peer (P2P)?? ",
                    "What is block chain?? ",
                    "What is private key?? "
                ]
            
            # Create pipeline
            print("\n🚀 Creating text generation pipeline...")
            pipe = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                device=0 if torch.cuda.is_available() else -1
            )
            
            # Run tests
            print("\n🧪 Running generation tests...")
            for i, prompt in enumerate(test_prompts, 1):
                print(f"\n🔹 Test {i}: {prompt}")
                output = pipe(
                    prompt,
                    max_length=max_length,
                    do_sample=True,
                    temperature=None,
                    top_p=None,
                    num_return_sequences=1,
                    repetition_penalty=1.2
                )
                print("💬 Response:", output[0]['generated_text'])
                
            return model, tokenizer
            
        except Exception as e:
            print(f"\n⚠️ Model loading failed: {str(e)}")
            print("🔄 Trying fallback loading method (local files only)...")
            
            model = AutoModelForCausalLM.from_pretrained(
                model_path,
                local_files_only=True
            )
            print("\n🎉 Fallback loading successful!")
            
            # Continue with testing as above
            # ... [rest of the testing code]
            
    except Exception as e:
        print(f"\n❌ Critical error loading model: {str(e)}")
        print("\n🛠️ Debugging info:")
        print(f"- Path: {os.path.abspath(model_path)}")
        print(f"- Directory exists: {os.path.exists(model_path)}")
        if os.path.exists(model_path):
            print("- Contents:", os.listdir(model_path))
        raise

In [27]:
from transformers import file_utils
file_utils.HF_DATASETS_CACHE = "/kaggle/working/cache"
file_utils.TRANSFORMERS_CACHE = "/kaggle/working/cache"

In [28]:
import gc
import torch

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

In [29]:
if __name__ == "__main__":
    model_path = "/kaggle/working/gpt2-lora-trained"
    
    # Save model artifacts
    save_model_artifacts(model, tokenizer, training_args)
    
    # Load with explicit path and PEFT flag
    load_and_test_model(model_path, is_peft_model=True)
    
    # Test with custom prompts
    custom_prompts = [
        " "
    ]
    load_and_test_model(model_path, test_prompts=custom_prompts, is_peft_model=True)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 50, 'do_sample': True}



💾 Saving model artifacts to: /kaggle/working/gpt2-lora-trained
💽 Saving model weights...
🔤 Saving tokenizer...
📝 Saving training arguments...

🔍 Verifying saved files:
- ✅ Model configuration (config.json)
- ✅ Tokenizer config (tokenizer_config.json)

⚖️ Found model weights:
- model.safetensors (474.71 MB)

🎉 All files saved successfully!

🔍 Preparing to load model from: /kaggle/working/gpt2-lora-trained

📂 Model directory contents:
- config.json (0.93 KB)
- generation_config.json (0.16 KB)
- merges.txt (445.62 KB)
- model.safetensors (486107.62 KB)
- special_tokens_map.json (0.10 KB)
- tokenizer.json (2058.38 KB)
- tokenizer_config.json (0.43 KB)
- training_args.json (3.86 KB)
- vocab.json (779.45 KB)

🔄 Loading tokenizer...

🔄 Loading model...

⚠️ Model loading failed: Can't find 'adapter_config.json' at '/kaggle/working/gpt2-lora-trained'
🔄 Trying fallback loading method (local files only)...

🎉 Fallback loading successful!

🔍 Preparing to load model from: /kaggle/working/gpt2-lora

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"  # Example of a supported model
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataset = prepare_dataset("/kaggle/input/database_0520.jsonl")  # Ensure dataset is loaded
text_samples = dataset["text"]  # Extract text list
text = text_samples[0]  # Select the first text entry for encoding

encoded_input = tokenizer(text, max_length=160, truncation=True, return_tensors="pt")

tensor = torch.randn(3, requires_grad=True)  # Enable gradients

❌ Dataset preparation failed: ❌ Path not found: /kaggle/input/database_0520.jsonl


In [34]:
# After successful loading
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [35]:
prompts = [
    "What is software wallet, and what's the difference between hardware and software wallet? ",
    "What is PoW? ",
    "Explain PoW in 1 sentence. ",
    "Describe the key features of PoW using 3 words. ",
    "What is PoM? Is it something related to cryptography? ",
    "What is a cryptographic product? ",
    "What is P2P? ",
    "What is block chain? ",
    "What is public key, and what's the difference between private and public key? "
]

for prompt in prompts:
    output = pipe(encoded_prompt["input_ids"], max_new_tokens=130, early_stopping=True, do_sample=False, top_p=None, temperature=None, repetition_penalty=1.2)
    print(f"Prompt: {prompt}\nResponse: {output[0]['generated_text']}\n")

NameError: name 'encoded_prompt' is not defined

In [ ]:
notebook_end = time.time()
print(f"Total notebook execution time: {notebook_end - notebook_start:.2f} seconds")